In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Start a Spark session with the additional option that adds the driver to Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("M16_Challenge_Vine-Data-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in video games dataset from S3 Buckets
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_games_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
# Drop null values
dropna_df = video_games_df.dropna()
dropna_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
# Drop duplicates
clean_df = dropna_df.drop_duplicates()
clean_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26197704|R10010D6B1QM7U|B004HX5OFW|       8155204|Xbox 360 Slim AC ...|     Video Games|          1|            0|          0|   N|                Y|                 POS|I purchased this ...| 2013-03-28|
|         US|   10409381|R100BI61LIKH2N|B000QW9D14|     583724133|Blue Dragon - Xbo...|     Video Games|          5|    

In [ ]:
# Create the Vine DataFrame
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10010D6B1QM7U|          1|            0|          0|   N|                Y|
|R100BI61LIKH2N|          5|            1|          2|   N|                Y|
|R100TPJCYDYGBU|          3|            1|          2|   N|                Y|
|R1018OO9U6UUWO|          5|            0|          0|   N|                Y|
|R102YIWZEIAXT9|          5|            2|          4|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter the Vine DataFrame by those that had 20+ total votes
df2 = vine_df.filter(vine_df["total_votes"] >= 20)
df2.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|
|R2NPZ79NDZ87XO|          1|            1|         31|   N|                N|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter the Vine DataFrame by those that had 50%+ of helpful votes
df3 = df2.filter(df2["helpful_votes"]/df2["total_votes"] >= 0.5)
df3.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter the Vine DataFrame by those that have a vine review
vine_yes_df = df3.filter(df3["vine"] == "Y")
vine_yes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R24PS9N276OXF4|          5|          241|        262|   Y|                N|
|R3EPHK0JGQGE91|          5|           21|         28|   Y|                N|
|R3T4LTJ189G0LT|          4|           37|         45|   Y|                N|
| RKG407AVAZXBQ|          3|           22|         25|   Y|                N|
| RNU8PK609WT6P|          4|          347|        362|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import col, avg

In [ ]:
# Get summary statistics of those that had vine reviews
vine_yes_df.describe().show()

+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|  count|            94|                94|               94|                94|  94|               94|
|   mean|          null| 4.202127659574468|54.59574468085106|61.787234042553195|null|             null|
| stddev|          null|0.9791348741656417|65.26098459822538| 68.90976994895392|null|             null|
|    min|R10FO5UKKVZBK2|                 1|               14|                20|   Y|                N|
|    max| RYBLWBAL5M1AY|                 5|              347|               362|   Y|                N|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+



In [ ]:
# Filter the Vine DataFrame by those that *do not* have a vine review
vine_no_df = df3.filter(df3["vine"] == "N")
vine_no_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Get summary statistics of those that did not have vine reviews
vine_no_df.describe().show()

+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|  count|         40471|             40471|             40471|             40471|40471|            40471|
|   mean|          null|  3.34765634651973|47.428405524943784|55.891057794470115| null|             null|
| stddev|          null|1.6418850112078018|117.53763370687125|127.40280622961963| null|             null|
|    min|R1005PFAPGMYQS|                 1|                10|                20|    N|                N|
|    max| RZZTUIX1574RC|                 5|             10498|             10780|    N|                Y|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+



In [ ]:
# Get percentage of 5-star rating among those records that had vine reviews
paid_five_star_number = vine_yes_df[vine_yes_df['star_rating']== 5].count()
paid_number = vine_yes_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

94
48
0.5106382978723404


In [ ]:
# Get percentage of 5-star rating among those record that did not have vine reviews
paid_five_star_number = vine_no_df[vine_no_df['star_rating']== 5].count()
paid_number = vine_no_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

40471
15663
0.38701786464381904
